**7.1 处理缺失数据**

- pandas对象的所有描述性统计默认都不包括缺失数据。
- 对于数值数据，pandas使用浮点值NaN（Not a Number）表示缺失数据。我们称其为哨兵值，可以方便的检测出来。
- 在pandas中，将缺失值表示为NA，它表示不可用not available。
- 在统计应用中，NA数据可能是不存在的数据或者虽然存在，但是没有观察到（例如，数据采集中发生了问题）。
- 当进行数据清洗以进行分析时，最好直接对缺失数据进行分析，以判断数据采集的问题或缺失数据可能导致的偏差。
- Python内置的None值在对象数组中也可以作为NA。

In [3]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])
string_data

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [4]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

In [5]:
string_data[0] = None
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

**滤除缺失数据**

对于一个Series，dropna返回一个仅含非空数据和索引值的Series

In [6]:
from numpy import nan as NA

data = pd.Series([1, NA, 3.5, NA, 7])
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

In [7]:
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

In [9]:
#dropna默认丢弃任何含有缺失值的行
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],
                    [NA, NA, NA], [NA, 6.5, 3.]])
cleaned = data.dropna()
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [10]:
cleaned

,0,1,2
0,1.0,6.5,3.0


In [11]:
#传入how='all'将只丢弃全为NA的那些行
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


In [12]:
data[4] = NA
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [13]:
data.dropna(axis=1, how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [14]:
# 用thresh参数实现留下一部分观测数据
df = pd.DataFrame(np.random.randn(7,3))
df.iloc[:4,1] = NA
df.iloc[:2,2] = NA
df

,0,1,2
0,-0.313693,NaN,NaN
1,0.049181,NaN,NaN
2,0.164996,NaN,-0.279883
3,-1.107128,NaN,-1.044564
4,1.877180,-2.575217,-1.253068
5,-0.480670,-1.336209,-0.352316
6,0.413870,0.496744,0.970774


In [15]:
df.dropna()

,0,1,2
4,1.87718,-2.575217,-1.253068
5,-0.48067,-1.336209,-0.352316
6,0.41387,0.496744,0.970774


In [16]:
df.dropna(thresh=2)

,0,1,2
2,0.164996,NaN,-0.279883
3,-1.107128,NaN,-1.044564
4,1.877180,-2.575217,-1.253068
5,-0.480670,-1.336209,-0.352316
6,0.413870,0.496744,0.970774


**填充缺失数据**

In [17]:
# fillna
df.fillna(0)

,0,1,2
0,-0.313693,0.000000,0.000000
1,0.049181,0.000000,0.000000
2,0.164996,0.000000,-0.279883
3,-1.107128,0.000000,-1.044564
4,1.877180,-2.575217,-1.253068
5,-0.480670,-1.336209,-0.352316
6,0.413870,0.496744,0.970774


In [18]:
#若是通过一个字典调用fillna，就可以实现对不同的列填充不同的值
df.fillna({1:0.5,2:0})

,0,1,2
0,-0.313693,0.500000,0.000000
1,0.049181,0.500000,0.000000
2,0.164996,0.500000,-0.279883
3,-1.107128,0.500000,-1.044564
4,1.877180,-2.575217,-1.253068
5,-0.480670,-1.336209,-0.352316
6,0.413870,0.496744,0.970774


In [19]:
#fillna默认会返回新对象，但也可以对现有对象进行就地修改
_ = df.fillna(0,inplace=True)
df

,0,1,2
0,-0.313693,0.000000,0.000000
1,0.049181,0.000000,0.000000
2,0.164996,0.000000,-0.279883
3,-1.107128,0.000000,-1.044564
4,1.877180,-2.575217,-1.253068
5,-0.480670,-1.336209,-0.352316
6,0.413870,0.496744,0.970774


In [20]:
#对reindexing有效的那些插值方法也可用于fillna
df = pd.DataFrame(np.random.randn(6,3))
df.iloc[2:,1] = NA
df.iloc[4:,2] = NA
df

,0,1,2
0,-1.369374,0.063183,0.460550
1,-0.054374,1.812216,-0.875737
2,-1.158242,NaN,0.132347
3,-0.453277,NaN,0.511975
4,0.216496,NaN,NaN
5,1.458271,NaN,NaN


In [21]:
df.fillna(method='ffill')

,0,1,2
0,-1.369374,0.063183,0.460550
1,-0.054374,1.812216,-0.875737
2,-1.158242,1.812216,0.132347
3,-0.453277,1.812216,0.511975
4,0.216496,1.812216,0.511975
5,1.458271,1.812216,0.511975


In [22]:
df.fillna(method='ffill', limit=2)

,0,1,2
0,-1.369374,0.063183,0.460550
1,-0.054374,1.812216,-0.875737
2,-1.158242,1.812216,0.132347
3,-0.453277,1.812216,0.511975
4,0.216496,NaN,0.511975
5,1.458271,NaN,0.511975


In [23]:
#可以传入Series的平均值或中位数
data = pd.Series([1., NA, 3.5, NA, 7])
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

**7.2 数据转换**

In [24]:
# 移除重复数据
data = pd.DataFrame({'k1':['one', 'two'] * 3 + ['two'], 
                    'k2': [1,1,2,3,3,4,4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


- DataFrame的duplicated方法返回一个布尔型Series，表示各行是否是重复行。
- drop_duplicates方法，它会返回一个DataFrame，重复的数组会标为False。
- 这两个方法默认会判断全部列，你也可以指定部分列进行重复项判断。
- duplicated和drop_duplicates默认保留的是第一个出现的值组合。传入keep='last'则保留最后一个。

In [26]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [27]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


In [28]:
data['v1'] = range(7)
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
1,two,1,1


In [29]:
data.drop_duplicates(['k1', 'k2'], keep='last')

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


**利用函数或映射进行数据转换**

In [30]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                             'Pastrami', 'corned beef', 'Bacon',
                             'pastrami', 'honey ham', 'nova lox'],
                    'ounces': [4,3,12,6,7.5,8,3,5,6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [31]:
meat_to_animal = {
    'bacon': 'pig',
    'pulled pork': 'pig',
    'pastrami': 'cow',
    'corned beef': 'cow',
    'honey ham': 'pig',
    'nova lox': 'salmon'
}

- Series的map方法可以接受一个函数或含有映射关系的字典型对象。
- 使用map是一种实现元素级转换以及其他数据清理工作的便捷方式。
- map可用于修改对象的数据子集。

In [32]:
lowercased = data['food'].str.lower()
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [33]:
data['animal'] = lowercased.map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [34]:
#传入一个能够完成全部这些工作的函数
data['food'].map(lambda x: meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

**替换值**

In [35]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [36]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [37]:
data.replace([-999, -1000], np.nan) #一次性替换多个值

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [38]:
#让每个值有不同的替换值，可以传递一个替换列表
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [39]:
#传入的参数也可以是字典
data.replace({-999: np.nan, -1000:0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

**重命名轴索引**

跟Series中的值一样，轴标签也可以通过函数或映射进行转换，从而得到一个新的不同标签的对象。轴还可以被就地修改，而无需新建一个数据结构。

In [41]:
data = pd.DataFrame(np.arange(12).reshape((3,4)),
                   index=['Ohio', 'Colorado', 'New York'],
                   columns=['one', 'two', 'three', 'four'])

transform = lambda x: x[:4].upper()
data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

In [42]:
data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [46]:
#如果想要创建数据集的转换版（而不是修改原始数据），比较实用的方法是rename
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


In [47]:
#rename可以结合字典型对象实现对部分轴标签的更新
data.rename(index={'OHIO': 'INDIANA'},
           columns={'three': 'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [48]:
#rename可以实现复制DataFrame并对其索引和列标签进行赋值。如果希望就地修改某个数据集，传入inplace=True即可
data.rename(index={'OHIO': 'INDIANA'}, inplace=True)
data

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


**离散化和面元划分**

为了便于分析，连续数据常常被离散化或拆分为“面元”（bin）。假设有一组人员数据，而你希望将它们划分为不同的年龄组。

In [49]:
ages = [20,22,25,27,21,23,37,31,61,45,41,32]

# 使用pandas中的cut函数,返回的是一个特殊的Categorical对象。
bins = [18, 25, 35, 60, 100]
cats = pd.cut(ages,bins)
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [50]:
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [51]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]],
              closed='right',
              dtype='interval[int64]')

In [52]:
#pd.value_counts(cats)是pandas.cut结果的面元计数。
pd.value_counts(cats)

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

In [53]:
#哪边是闭端可以通过right=False进行修改
pd.cut(ages, [18,26,36,61,100],right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

In [54]:
#通过传递一个列表或数组到labels，设置自己的面元名称
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names)

[Youth, Youth, Youth, YoungAdult, Youth, ..., YoungAdult, Senior, MiddleAged, MiddleAged, YoungAdult]
Length: 12
Categories (4, object): [Youth < YoungAdult < MiddleAged < Senior]

In [56]:
data = np.random.rand(20)
pd.cut(data, 4, precision=2, right=False) #选项precision=2，限定小数只有两位。

[[0.026, 0.24), [0.24, 0.46), [0.68, 0.9), [0.68, 0.9), [0.24, 0.46), ..., [0.46, 0.68), [0.46, 0.68), [0.24, 0.46), [0.24, 0.46), [0.68, 0.9)]
Length: 20
Categories (4, interval[float64]): [[0.026, 0.24) < [0.24, 0.46) < [0.46, 0.68) < [0.68, 0.9)]

- qcut是一个非常类似于cut的函数，它可以根据样本分位数对数据进行面元划分。
- 根据数据的分布情况，cut可能无法使各个面元中含有相同数量的数据点。
- 而qcut由于使用的是样本分位数，因此可以得到大小基本相等的面元。

In [59]:
data = np.random.randn(1000) # Normally distributed
cats = pd.qcut(data, 4) # Cut into quartiles
cats

[(-0.745, -0.0617], (-3.09, -0.745], (-3.09, -0.745], (-0.0617, 0.658], (0.658, 3.851], ..., (-0.745, -0.0617], (0.658, 3.851], (-0.745, -0.0617], (-0.0617, 0.658], (0.658, 3.851]]
Length: 1000
Categories (4, interval[float64]): [(-3.09, -0.745] < (-0.745, -0.0617] < (-0.0617, 0.658] < (0.658, 3.851]]

In [60]:
pd.value_counts(cats)

(0.658, 3.851]       250
(-0.0617, 0.658]     250
(-0.745, -0.0617]    250
(-3.09, -0.745]      250
dtype: int64

In [61]:
#可以传递自定义的分位数（0到1之间的数值，包含端点）
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])

[(-1.343, -0.0617], (-1.343, -0.0617], (-1.343, -0.0617], (-0.0617, 1.254], (-0.0617, 1.254], ..., (-1.343, -0.0617], (-0.0617, 1.254], (-1.343, -0.0617], (-0.0617, 1.254], (1.254, 3.851]]
Length: 1000
Categories (4, interval[float64]): [(-3.09, -1.343] < (-1.343, -0.0617] < (-0.0617, 1.254] < (1.254, 3.851]]

**检测和过滤异常值**

过滤或变换异常值（outlier）在很大程度上是运用数组运算。

In [62]:
data = pd.DataFrame(np.random.randn(1000,4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.021885,-0.079927,-0.003237,0.004374
std,1.004423,0.990497,1.032756,0.991233
min,-3.000073,-3.198970,-3.801726,-3.309662
25%,-0.679280,-0.722473,-0.679858,-0.658414
50%,0.075065,-0.047614,-0.014935,-0.022225
75%,0.736390,0.563482,0.706700,0.681290
max,2.717234,3.112054,3.415971,2.953938


In [63]:
#找出某列中绝对值大小超过3的值
col = data[2]
col[np.abs(col) > 3]

47    -3.204334
57    -3.242499
241   -3.801726
872    3.415971
917    3.157285
Name: 2, dtype: float64

In [64]:
#要选出全部含有“超过3或－3的值”的行，可在布尔型DataFrame中使用any方法
data[(np.abs(data) > 3).any(1)]

,0,1,2,3
37,-3.000073,-0.045418,0.312332,0.124276
47,-2.047348,0.557201,-3.204334,-0.547829
57,-0.758115,-0.627997,-3.242499,1.296966
111,0.856376,-3.198970,-0.665717,-0.051938
241,-0.068103,1.478671,-3.801726,-0.037892
442,-0.640681,0.304250,1.060122,-3.309662
444,0.001638,-3.013675,-0.882574,-0.469260
488,1.036834,3.043996,-0.253753,-1.019046
538,-1.009250,3.059809,0.014478,1.568034
787,1.299794,3.112054,1.049914,-1.137219


In [65]:
data[np.abs(data)>3] = np.sign(data) * 3 #将值限制在区间－3到3以内
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.021885,-0.079930,-0.002562,0.004684
std,1.004423,0.989167,1.026989,0.990244
min,-3.000000,-3.000000,-3.000000,-3.000000
25%,-0.679280,-0.722473,-0.679858,-0.658414
50%,0.075065,-0.047614,-0.014935,-0.022225
75%,0.736390,0.563482,0.706700,0.681290
max,2.717234,3.000000,3.000000,2.953938


In [66]:
#根据数据的值是正还是负，np.sign(data)可以生成1和-1
np.sign(data).head()

,0,1,2,3
0,1.0,-1.0,1.0,1.0
1,1.0,-1.0,1.0,1.0
2,-1.0,-1.0,-1.0,1.0
3,1.0,1.0,-1.0,-1.0
4,-1.0,1.0,1.0,-1.0


**排列和随机采样**

利用numpy.random.permutation函数可以轻松实现对Series或DataFrame的列的排列工作（permuting，随机重排序）。通过需要排列的轴的长度调用permutation，可产生一个表示新顺序的整数数组

In [67]:
df = pd.DataFrame(np.arange(5*4).reshape((5,4)))
sampler = np.random.permutation(5)
sampler

array([0, 2, 3, 1, 4])

In [68]:
#在基于iloc的索引操作或take函数中使用该数组
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [69]:
df.take(sampler)

,0,1,2,3
0,0,1,2,3
2,8,9,10,11
3,12,13,14,15
1,4,5,6,7
4,16,17,18,19


In [71]:
#如果不想用替换的方式选取随机子集，可以在Series和DataFrame上使用sample方法
df.sample(n=3)

,0,1,2,3
3,12,13,14,15
0,0,1,2,3
4,16,17,18,19


In [73]:
#通过替换的方式产生样本（允许重复选择），可以传递replace=True到sample
choices = pd.Series([5,7,-1,6,4])
draws = choices.sample(n=10, replace=True)
draws

0    5
3    6
1    7
0    5
1    7
4    4
3    6
0    5
4    4
1    7
dtype: int64

**计算指标/哑变量**

- 另一种常用于统计建模或机器学习的转换方式是：将分类变量（categorical variable）转换为“哑变量”或“指标矩阵”。
- 如果DataFrame的某一列中含有k个不同的值，则可以派生出一个k列矩阵或DataFrame（其值全为1和0）。
- pandas有一个get_dummies函数可以实现该功能（其实自己动手做一个也不难）。

In [76]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                  'data1': range(6)})
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [77]:
#get_dummies的prefix参数可以实现:给指标DataFrame的列加上一个前缀，以便能够跟其他数据进行合并。
dummies = pd.get_dummies(df['key'], prefix='key')
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


In [80]:
#如果DataFrame中的某行同属于多个分类，则事情就会有点复杂。
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_csv('datasets\movielens\movies.dat', sep='::',
                      header=None, names=mnames)
movies[:10]

E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  after removing the cwd from sys.path.


,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [82]:
#我们从数据集中抽取出不同的genre值
all_genres = []
for x in movies.genres:
    all_genres.extend(x.split('|'))
genres = pd.unique(all_genres)
genres

array(['Animation', "Children's", 'Comedy', 'Adventure', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Sci-Fi', 'Documentary', 'War', 'Musical', 'Mystery', 'Film-Noir',
       'Western'], dtype=object)

In [83]:
#构建指标DataFrame的方法之一是从一个全零DataFrame开始
zero_matrix = np.zeros((len(movies), len(genres)))
dummies = pd.DataFrame(zero_matrix, columns=genres)

In [84]:
#迭代每一部电影，并将dummies各行的条目设为1。
gen = movies.genres[0]
gen.split('|')

['Animation', "Children's", 'Comedy']

In [85]:
dummies.columns.get_indexer(gen.split('|'))

array([0, 1, 2], dtype=int64)

In [90]:
# 根据索引，使用.iloc设定值
for i, gen in enumerate(movies.genres):
    indices = dummies.columns.get_indexer(gen.split('|'))
    dummies.iloc[i, indices] = 1

In [91]:
movies_windic = movies.join(dummies.add_prefix('Genre_'))
movies_windic.iloc[0]

movie_id                                       1
title                           Toy Story (1995)
genres               Animation|Children's|Comedy
Genre_Animation                                1
Genre_Children's                               1
Genre_Comedy                                   1
Genre_Adventure                                0
Genre_Fantasy                                  0
Genre_Romance                                  0
Genre_Drama                                    0
Genre_Action                                   0
Genre_Crime                                    0
Genre_Thriller                                 0
Genre_Horror                                   0
Genre_Sci-Fi                                   0
Genre_Documentary                              0
Genre_War                                      0
Genre_Musical                                  0
Genre_Mystery                                  0
Genre_Film-Noir                                0
Genre_Western       

- 对于很大的数据，用这种方式构建多成员指标变量就会变得非常慢。最好使用更低级的函数，将其写入NumPy数组，然后结果包装在DataFrame中。
- 对统计应用有用的秘诀是：结合get_dummies和诸如cut之类的离散化函数

In [92]:
np.random.seed(12345)
values = np.random.rand(10)
values

array([0.92961609, 0.31637555, 0.18391881, 0.20456028, 0.56772503,
       0.5955447 , 0.96451452, 0.6531771 , 0.74890664, 0.65356987])

In [93]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
pd.get_dummies(pd.cut(values, bins))

   (0.0, 0.2]  (0.2, 0.4]  (0.4, 0.6]  (0.6, 0.8]  (0.8, 1.0]
0           0           0           0           0           1
1           0           1           0           0           0
2           1           0           0           0           0
3           0           1           0           0           0
4           0           0           1           0           0
5           0           0           1           0           0
6           0           0           0           0           1
7           0           0           0           1           0
8           0           0           0           1           0
9           0           0           0           1           0

**7.3 字符串操作**

**字符串对象方法**

In [94]:
val = 'a,b, guido'
val.split(',')

['a', 'b', ' guido']

In [95]:
#split常常与strip一起使用，以去除空白符（包括换行符）
pieces = [x.strip() for x in val.split(',')]
pieces

['a', 'b', 'guido']

In [96]:
first, second, third = pieces
first + '::' + second + '::' + third

'a::b::guido'

In [97]:
'::'.join(pieces)

'a::b::guido'

In [98]:
#子串定位。检测子串的最佳方式是利用Python的in关键字，还可以使用index和find
'guido' in val

True

In [99]:
val.index(',')

1

In [100]:
#注意find和index的区别：如果找不到字符串，index将会引发一个异常（而不是返回－1）
val.find(':')

-1

In [101]:
val.index(':')

ValueError: substring not found

In [102]:
val.count(',')#count可以返回指定子串的出现次数

2

In [103]:
val.replace(',', '::')

'a::b:: guido'

In [104]:
val.replace(',', '')

'ab guido'

In [105]:
val.endswith('a')

False

In [106]:
val.startswith('a')

True

In [108]:
val.rfind('u')

6

**正则表达式**

- 正则表达式提供了一种灵活的在文本中搜索或匹配（通常比前者复杂）字符串模式的方式。
- 正则表达式，常称作regex，是根据正则表达式语言编写的字符串。
- Python内置的re模块负责对字符串应用正则表达式。
- re模块的函数可以分为三个大类：模式匹配、替换以及拆分。
- 如果打算对许多字符串应用同一条正则表达式，强烈建议通过re.compile创建regex对象。这样将可以节省大量的CPU时间。
- findall返回的是字符串中所有的匹配项;
- search则只返回第一个匹配项。
- match更加严格，它只匹配字符串的首部。

In [109]:
import re
text = "foo    bar\t  baz  \t qux"
re.split('\s+', text) #描述一个或多个空白符的regex是\s+

['foo', 'bar', 'baz', 'qux']

In [110]:
#用re.compile自己编译regex以得到一个可重用的regex对象
regex = re.compile('\s+')
regex.split(text)

['foo', 'bar', 'baz', 'qux']

In [111]:
#只希望得到匹配regex的所有模式，则可以使用findall方法
regex.findall(text)

['    ', '\t  ', '  \t ']

In [112]:
text = """Deve dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'

#re.IGNORECASE makes the regex case-insensitive
regex = re.compile(pattern, flags=re.IGNORECASE)

In [113]:
#对text使用findall将得到一组电子邮件地址
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

In [115]:
m = regex.search(text)
m

<_sre.SRE_Match object; span=(5, 20), match='dave@google.com'>

In [116]:
text[m.start():m.end()]

'dave@google.com'

In [117]:
#regex.match则将返回None，因为它只匹配出现在字符串开头的模式
print(regex.match(text))

None


In [118]:
#sub方法可以将匹配到的模式替换为指定字符串，并返回所得到的新字符串
print(regex.sub('REDACTED', text))

Deve REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED



In [120]:
#将各个地址分成3个部分：用户名、域名以及域后缀。
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'
regex = re.compile(pattern, flags=re.IGNORECASE)

m = regex.match('wesm@bright.net')
m.groups()

('wesm', 'bright', 'net')

In [121]:
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

sub还能通过诸如\1、\2之类的特殊符号访问各匹配项中的分组。符号\1对应第一个匹配的组，\2对应第二个匹配的组，以此类推

In [123]:
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text))

Deve Username: dave, Domain: google, Suffix: com
Steve Username: steve, Domain: gmail, Suffix: com
Rob Username: rob, Domain: gmail, Suffix: com
Ryan Username: ryan, Domain: yahoo, Suffix: com



**pandas的矢量化字符串函数**

In [124]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
       'Rob': 'rob@gmail.com', 'Wes': np.nan}
data = pd.Series(data)
data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

In [125]:
data.isnull()

Dave     False
Steve    False
Rob      False
Wes       True
dtype: bool

In [126]:
#通过str.contains检查各个电子邮件地址是否含有"gmail"
data.str.contains('gmail')

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

In [127]:
pattern 

'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\\.([A-Z]{2,4})'

In [128]:
data.str.findall(pattern, flags=re.IGNORECASE)

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

In [129]:
#可以实现矢量化的元素获取操作：要么使用str.get，要么在str属性上使用索引
matches = data.str.match(pattern, flags=re.IGNORECASE)
matches

Dave     True
Steve    True
Rob      True
Wes       NaN
dtype: object

In [130]:
#要访问嵌入列表中的元素，我们可以传递索引到这两个函数中
matches.str.get(1)

Dave    NaN
Steve   NaN
Rob     NaN
Wes     NaN
dtype: float64

In [131]:
matches.str[0]

Dave    NaN
Steve   NaN
Rob     NaN
Wes     NaN
dtype: float64

In [132]:
data.str[:5]

Dave     dave@
Steve    steve
Rob      rob@g
Wes        NaN
dtype: object